# DLMDSPWP01 - Calculation

In this Jupyter Notebook, we implement a Python program to handle and analyze training datasets stored in CSV format. The primary objective is to construct an SQLite database using sqlalchemy, where the training data is organized into a five-column table.

Additionally, we load fifty ideal functions from a separate CSV file into another table within the database. This table features 51 columns, with the first column representing x-values.

After successfully populating the database with training data and ideal functions,the program proceeds to process test data (B) from yet another CSV file. Each line of test data is loaded and, if it meets specified criteria, matched with one of four chosen functions from the previous subsection (i). The results are then stored in a dedicated four-column table in the SQLite database.

This notebook only finds the best ideal functions and stores the points that fulfill the criteria into the SQL Database. For an implementation of Pandas and Bokeh, please take a look at [visualize.ipynb](./visualize.ipynb)

## Pre-Requesites
The next cell loads all of the requirements needed for this notebook. For a setup guide (for MacOSX) please consult the [README](./README.md)

In [13]:
import os

from sqlalchemy import Column, inspect

from entities.point import Point
from regressor.regressor import Regressor
from services.sql_database import SQLDatabase
from sqlalchemy import Numeric


## Loading the Data into the database
The next view cells, create a SQLite database and loads the train dataset and the ideal functions into two different tables. The data is loaded from the local file system.

In [14]:
sql_database = SQLDatabase()

In [15]:
# Load the training data
columns = [Column(column_name, type_=Numeric) for column_name in ["X"] + ["Y" + str(i) for i in range(1, 5)]]
sql_database.create_table("train", columns=columns)
sql_database.truncate_table("train")
with open(os.path.join("data", "train", "train.csv"), "r") as file:
    file.readline()  # Skip first line
    for i, row in enumerate(file.readlines()):
        sql_database.insert_into_table("train", values=row.split(","), column_names=columns)

In [16]:
# Test the training data is loaded correctly
assert "train" in inspect(sql_database.engine).get_table_names()
with sql_database.engine.connect() as connection:
    query_result = connection.execute(f"SELECT * FROM 'train'").fetchall()
    with open(os.path.join("data", "train", "train.csv"), "r") as file:
        table_values = [row.strip("\n").split(",") for i, row in enumerate(file.readlines()) if i]
        assert list(map(list, query_result)) == [list(map(float, row)) for row in table_values]

In [17]:
# Load the ideal data
columns = [Column(column_name, type_=Numeric) for column_name in ["X"] + ["Y" + str(i) for i in range(1, 51)]]
sql_database.create_table("ideal", columns=columns)
sql_database.truncate_table("ideal")
with open(os.path.join("data", "ideal", "ideal.csv"), "r") as file:
    file.readline()  # Skip first line
    for i, row in enumerate(file.readlines()):
        sql_database.insert_into_table("ideal", values=row.split(","), column_names=columns)

In [18]:
# Test the ideal data is loaded correctly
assert "ideal" in inspect(sql_database.engine).get_table_names()
with sql_database.engine.connect() as connection:
    query_result = connection.execute(f"SELECT * FROM 'ideal'").fetchall()
    with open(os.path.join("data", "ideal", "ideal.csv"), "r") as file:
        table_values = [row.strip("\n").split(",") for i, row in enumerate(file.readlines()) if i]
        assert list(map(list, query_result)) == [list(map(float, row)) for row in table_values]

## Finding the ideal functions

In [19]:
regressor = Regressor()
best_ideal_functions = []
for dataset_id in range(1, 5):
    with sql_database.engine.connect() as connection:
        train_sql_query = f"SELECT x, y{dataset_id} FROM 'train'"
        train_result = connection.execute(train_sql_query).fetchall()
        train_points = [Point(item[0], item[1]) for item in train_result]
        
        ideal_sql_query = f"SELECT * FROM 'ideal'"
        ideal_result = connection.execute(ideal_sql_query).fetchall()
        ideal_points = []
        for item in ideal_result:
            for i in range(len(item)-1):
                x = item[0]
                y = item[1 + i]
                if 1 + i > len(ideal_points):
                    ideal_points.append([])
                ideal_points[i].append(Point(x, y))

        best_ideal_function, max_sum, max_deviation = regressor.find_ideal_function(train_points, ideal_points)
        best_ideal_functions.append((best_ideal_function, max_sum, max_deviation))

In [20]:
best_ideal_functions = sorted(best_ideal_functions, key=lambda x: x[0])
best_ideal_functions

[(8, 99.99999945140036, 0.25000010000000994),
 (14, 99.99999999000272, 0.250001000001001),
 (24, 99.99999963000016, 0.2500000600000036),
 (35, 99.99999998000058, 0.2500004000001602)]

## Testing the best ideal functions on the test dataset

In [21]:
best_ideal_function_lookup = {i: best_ideal_functions[i] for i in range(len(best_ideal_functions))}
resulting_points = []
with open(os.path.join("data", "test", "test.csv"), "r", encoding="utf-8") as test_file:
    test_file.readline()
    for line in test_file.readlines():
        item = line.split(",")
        x = float(item[0])
        y = float(item[1].replace("\\n", ""))
        test_point = Point(x, y)
        with sql_database.engine.connect() as connection:
            sql_query = f"SELECT x, y{best_ideal_functions[0][0]}, y{best_ideal_functions[1][0]}, y{best_ideal_functions[2][0]}, y{best_ideal_functions[3][0]} FROM 'ideal' WHERE x={test_point.x}"
            query_result = connection.execute(sql_query).fetchall()
            assert len(query_result) == 1
            ideal_points = [
                Point(query_result[0][0], query_result[0][1]),
                Point(query_result[0][0], query_result[0][2]),
                Point(query_result[0][0], query_result[0][3]),
                Point(query_result[0][0], query_result[0][4]),
            ]
            
            differences = [
                (i, test_point.calculate_least_square_difference(other_point)) for i, other_point in enumerate(ideal_points)
            ]
            sorted_differences = sorted(differences, key=lambda x: x[1])
            ideal_function = best_ideal_function_lookup[sorted_differences[0][0]]
            
            # assert max_deviation_of_ideal_function <= sorted_differences[0][1] * 2 ** 0.5
            if ideal_function[2] <= sorted_differences[0][1] * 2 ** 0.5:
                resulting_points.append((test_point.x, test_point.y, ideal_function[1], ideal_function[0]))        
            

In [22]:
columns=[
    Column("x", type_=Numeric),
    Column("y", type_=Numeric),
    Column("delta_y", type_=Numeric),
    Column("number", type_=Numeric)
]
sql_database.create_table("test", columns=columns)
sql_database.truncate_table("test")
for result in resulting_points:
    sql_database.insert_into_table("test", values=list(map(str, result)), column_names=columns)

In [23]:
with sql_database.engine.connect() as connection:
    sql_query = "SELECT * from test"
    query_result = connection.execute(sql_query).fetchall()
    print(query_result)

[(-16.5, -8983.502, 99.99999963000016, 24), (-9.4, -1.6981516, 99.99999998000058, 35), (-18.5, -12662.127, 99.99999963000016, 24), (1, -0.116328545, 99.99999999000272, 14), (16, -2.352354, 99.99999945140036, 8), (-12, -3454.9302, 99.99999963000016, 24), (0.8, -8.55054, 99.99999998000058, 35), (1.8, -2.80888, 99.99999999000272, 14), (3, -7.6883636, 99.99999998000058, 35), (-1.1, -1817.7866, 99.99999998000058, 35), (-12.7, -4095.8215, 99.99999963000016, 24), (-18.4, 0.19691324, 99.99999945140036, 8), (3, 53.231857, 99.99999963000016, 24), (8.6, -8.133972, 99.99999999000272, 14), (-17.2, 6.764539, 99.99999998000058, 35), (-2.6, -8.025898, 99.99999998000058, 35), (5.2, -15348.265, 99.99999999000272, 14), (-15.5, 6.221319, 99.99999998000058, 35), (-0.3, 0.7445385, 99.99999999000272, 14), (-1.2, 1.6692932, 99.99999999000272, 14), (18.8, 7.4445, 99.99999998000058, 35), (-3, -53.02533, 99.99999963000016, 24), (-14.9, 1.7203728, 99.99999945140036, 8), (-17.3, 6.355572, 99.99999998000058, 35), (